In [1]:
import pandas as pd
import sys
# sys.path.append('I:/YaoTony/code/x8313')
from datetime import date
import pytest
# from experiments.fairvalue

In [20]:
def compute_simple_payout(df, attr_model, bet_amount=1.0):
    """
    Add columns for quick calculation of Win bets % payout
    :param df: Dataframe from dataset for multiple races
    :param attr_model: (string)an attribute / prob or score in the dataframe that can be ranked
    :param ascending: False if best is higher score i.e. probs
    :param bet_amount: default to 1.0
    :return: Dataframe with columns added

    """
    df['is_win'] = df['official_finish_position'].map(lambda x:int(x==1))
    df['rank_' + attr_model] = df.groupby('race_id')[attr_model].transform(lambda x: x.rank(ascending=False))
    df['bet_amount'] = df['rank_' + attr_model].map(lambda x:int(x<1.5)*bet_amount)
    df['is_wager'] = df['bet_amount'].map(lambda x:int(x>0))
    df['is_paid'] = df['is_wager'] * df['is_win']
    df['payout'] = df['is_win']*df['bet_amount']*df['payout_win'].fillna(0.0)

    return df

In [5]:
def compute_advantage(df):
    try:
        #print(df.race_id).iloc[0]
        pct_win = df.groupby('race_id')['is_paid'].sum().value_counts(normalize=True)[1]
        
    except:
        
        return 0.0

    pct_loss = 1.0 - pct_win
    mean_odds = df[df.is_paid>0]['final_tote_odds'].mean()
    advantage = pct_win - pct_loss / mean_odds
    #print(advantage)
    return advantage

In [6]:
import os
from os import path

In [15]:
df_factors_PILOT.head()

,race_id,track_id,date,race_number,distance,approx_dist,surface,race_type,sex_restriction,age_restriction,...,x8rank_speed_2of3,x8speed_norm_2of3_par,x8is_longshot,x8is_win_longshot,x8is_exacta_longshot,x8is_trifecta_lonsghot,x8is_superfecta_longshot,x8_outperform,pct_of_purse_earnings,runner_program_number_pp
0,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,3.0,0.614865,0,0.0,0.0,0.0,0.0,-2.0,0.015,7
1,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,5.0,0.486486,1,0.0,0.0,1.0,1.0,1.0,0.100,4
2,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,6.0,0.418919,1,0.0,0.0,0.0,0.0,1.0,0.000,5
3,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,2.0,0.743243,0,0.0,0.0,0.0,0.0,0.0,0.600,2
4,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,1.0,0.770270,0,0.0,0.0,0.0,0.0,0.0,0.250,6


In [26]:
df_factors_PILOT = pd.read_csv('./df_factors_PILOT.csv')
#df_factors_PILOT = pd.read_csv('I:\\YaoTony\\saleem tasks\\df_factors_PILOT.csv')
# we do not have payout_win column in the original file. We created one using payout = final odds+1
df_factors_PILOT['payout_win'] = df_factors_PILOT['final_tote_odds'] + 1
#factors we want to calculate advantage
factors = ['AlternateSpeedPoints','BestHDWSpeedAllWeatherSurface','HDWPSRRating','NumericHDWRunStyleDesignation',
           'SpeedPoints','bestCramerSpeedFigDirt','riderROI','trainerROI','x8runner_HDWPSRRating_norm_par','x8speed_norm_2of3_par',
           'x8ratio_earnings_0_claim']
[x in list(df_factors_PILOT.columns)for x in factors]

[True, True, True, True, True, True, True, True, True, True, True]

In [27]:
df_factors_PILOT.reset_index(inplace=True)


In [32]:
df_factors_PILOT = compute_simple_payout(df_factors_PILOT, 'x8speed_norm_2of3_par')
df_factors_PILOT.groupby('race_id')['is_paid'].sum().value_counts(normalize=True)

0    0.728294
1    0.271706
Name: is_paid, dtype: float64

In [4]:
# ranking according to every factor, descending
for factor in factors:
    df_factors_PILOT['rank_' + factor] = df_factors_PILOT.groupby("race_id")[factor].transform(lambda x:x.rank(ascending=False))
#df_factors_PILOT.to_csv('./df_factors_PILOT_rank.csv')

In [5]:
# create a dictionary of advantages. Keys are factor names
def compute_advantage_by_group(df, grouping_columns):
    """
    groups by grouping columns and computes advantages
    """
    grouped = df.groupby(grouping_columns)
    grp_dict = {}
    for grp, dfgrp in grouped:
        advantage = {}
        for factor in factors:
            df = dfgrp.copy()
            attr_bench_final = factor
            df_new = compute_simple_payout(df, attr_model=attr_bench_final, ascending=False, bet_amount=1.0).copy()
            advantage[factor] = compute_advantage(df_new)
        grp_dict[grp] = advantage
    return grp_dict


In [6]:
#group races by num_starters
grp_num_starters_pp = df_factors_PILOT.groupby('x8_num_starters_pp')


In [7]:
grp_num_starters_pp['race_id'].unique().apply(lambda x:len(x))

x8_num_starters_pp
1       7
2      13
3      10
4      19
5      85
6     167
7     215
8     164
9     141
10     73
11     36
12     25
13     12
14      8
15      4
Name: race_id, dtype: int64

In [8]:
criteria_trifecta_base = lambda df:df['x8_num_starters_pp'].isin([8,9,10,11])


In [9]:
def add_agg_factor_column(df, grpby_columns, factor_column, aggfunc, prefix):
    #Adds a column to df by applying an agg function to the factor by group
    df["x8agg"+"_"+factor_column+prefix] = df.groupby(grpby_columns)[factor_column].transform(aggfunc)
    return df

In [10]:
df_factors_PILOT = add_agg_factor_column(df_factors_PILOT, grpby_columns='race_id',factor_column='x8is_HDWPSRRating_norm_par', aggfunc=sum, prefix='sum')

In [11]:
df_factors_PILOT.drop(['level_0'], axis=1, inplace=True)

ValueError: labels ['level_0'] not contained in axis

In [ ]:
df_factors_PILOT.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)

In [ ]:
df_factors_PILOT_bygroup = df_factors_PILOT.copy()

In [12]:
df_factors_PILOT_bygroup.to_csv('./df_factors_PILOT_bygroup.csv', index=False)

NameError: name 'df_factors_PILOT_bygroup' is not defined

In [13]:
df['num_par_HDWPSRRating'] = df.groupby(['race_id'])['x8is_HDWPSRRating_norm_par'].transform(sum)

NameError: name 'df' is not defined

In [22]:
dict_factors = compute_advantage_by_group(df_factors_PILOT, grouping_columns=['x8_num_starters_pp','x8is_HDWPSRRating_norm_par'])

In [23]:
grp_distance_surface_runners = df_factors_PILOT.groupby(['distance','surface','x8_num_starters_pp'])#.apply(lambda df:compute_advantage_by_group(df, grouping_columns=['x8is_HDWPSRRating_norm_par']))

In [24]:
grp_distance_surface_runners.first()

race_id track_id        date  \
distance surface x8_num_starters_pp                                         
770.0    D       1                    NPX_20170707_3      NPX  2017-07-07   
                 4                    NPX_20170703_4      NPX  2017-07-03   
880.0    D       1                    PEN_20170708_1      PEN  2017-07-08   
                 5                    FMT_20170707_3      FMT  2017-07-07   
                 6                    FMT_20170708_2      FMT  2017-07-08   
                 9                    TDN_20170705_5      TDN  2017-07-05   
990.0    A       2                    APX_20170707_6      APX  2017-07-07   
         D       1                    EMD_20170703_1      EMD  2017-07-03   
                 2                    LAD_20170705_1      LAD  2017-07-05   
                 6                    LAX_20170707_4      LAX  2017-07-07   
                 8                    CTX_20170706_3      CTX  2017-07-06   
                 9                    CTX_20170708_6      CTX  2017-07-08   
                 10                   CTX_20170706_1      CTX  2017-07-06   
                 13                   CTX_20170708_8      CTX  2017-07-08   
1100.0   A       4                    PID_20170704_3      PID  2017-07-04   
                 6                    PID_20170705_2      PID  2017-07-05   
         D       1                    CBY_20170703_5      CBY  2017-07-03   
                 2                    PLN_20170704_3      PLN  2017-07-04   
                 3                    DEL_20170705_3      DEL  2017-07-05   
                 4                    CBY_20170704_3      CBY  2017-07-04   
                 5                    CBY_20170703_1      CBY  2017-07-03   
                 6                    FLX_20170703_2      FLX  2017-07-03   
                 7                    CBY_20170703_3      CBY  2017-07-03   
                 8                    LSX_20170704_9      LSX  2017-07-04   
                 9                    ELP_20170703_7      ELP  2017-07-03   
                 10                   ELP_20170704_4      ELP  2017-07-04   
         T       6                    APX_20170703_2      APX  2017-07-03   
                 7                    DEL_20170703_8      DEL  2017-07-03   
                 8                    FEX_20170704_3      FEX  2017-07-04   
                 9                    LSX_20170703_1      LSX  2017-07-03   
...                                              ...      ...         ...   
1870.0   T       6                    BEL_20170703_5      BEL  2017-07-03   
                 7                    ELP_20170707_3      ELP  2017-07-07   
                 8                    APX_20170703_5      APX  2017-07-03   
                 9                    APX_20170703_7      APX  2017-07-03   
                 10                   CBY_20170703_9      CBY  2017-07-03   
                 11                   LAD_20170703_6      LAD  2017-07-03   
                 12                   PHA_20170704_8      PHA  2017-07-04   
                 13                   LAD_20170704_6      LAD  2017-07-04   
                 14                   ELP_20170708_8      ELP  2017-07-08   
         t       7                    BEL_20170704_6      BEL  2017-07-04   
                 8                    BEL_20170703_2      BEL  2017-07-03   
                 9                    BEL_20170707_3      BEL  2017-07-07   
                 10                   BEL_20170709_6      BEL  2017-07-09   
                 13                   BEL_20170708_5      BEL  2017-07-08   
1980.0   D       8                    BTP_20170708_4      BTP  2017-07-08   
                 10                   CTX_20170707_4      CTX  2017-07-07   
         T       7                    APX_20170708_5      APX  2017-07-08   
                 9                    SAX_20170704_3      SAX  2017-07-04   
2090.0   A       9                    PID_20170706_3      PID  2017-07-06   
         T       8                    APX

In [28]:
compute_advantage_by_group(df_factors_PILOT, grouping_columns=['x8is_HDWPSRRating_norm_par','post_position' ])

{(0, 1.0): {'AlternateSpeedPoints': -0.017619012535390624,
  'BestHDWSpeedAllWeatherSurface': -0.017619012535390624,
  'HDWPSRRating': -0.017619012535390624,
  'NumericHDWRunStyleDesignation': -0.017619012535390624,
  'SpeedPoints': -0.017619012535390624,
  'bestCramerSpeedFigDirt': -0.017619012535390624,
  'riderROI': -0.017619012535390624,
  'trainerROI': -0.017619012535390624,
  'x8ratio_earnings_0_claim': -0.029576280723122864,
  'x8runner_HDWPSRRating_norm_par': -0.018889978637933097,
  'x8speed_norm_2of3_par': -0.018889978637933097},
 (0, 2.0): {'AlternateSpeedPoints': -0.060778440628277378,
  'BestHDWSpeedAllWeatherSurface': -0.060778440628277378,
  'HDWPSRRating': -0.060778440628277378,
  'NumericHDWRunStyleDesignation': -0.060778440628277378,
  'SpeedPoints': -0.060778440628277378,
  'bestCramerSpeedFigDirt': -0.060778440628277378,
  'riderROI': -0.060778440628277378,
  'trainerROI': -0.060778440628277378,
  'x8ratio_earnings_0_claim': -0.1133907607326133,
  'x8runner_HDWPSRRa

In [26]:
df_factors_PILOT.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7406 entries, 0 to 7405
Data columns (total 264 columns):
race_id                                object
track_id                               object
date                                   object
race_number                            int64
distance                               float64
approx_dist                            bool
surface                                object
race_type                              object
sex_restriction                        object
age_restriction                        object
age_limit                              object
race_classification                    object
purse                                  int64
claiming_price                         float64
track_record                           float64
race_conditions                        object
breed_type                             object
full_race_conditions                   object
bet_description                        object
low_claiming_price 

In [27]:
advantage


NameError: name 'advantage' is not defined